# **Time series analysis of flight delays**

# Introduction

This project aims at performing a time series analysis to forecast the departure delays in the US from 2019 to 2023 and presents with an opportunity to analyse the challenges and enhance the operational efficiency and passenger satisfaction in the aviation industry. The project begins with a comprehensive examination of historical flight data, focusing on the `DEP_DELAY` variable that represents the length of delay at the time of departure. The data is preprocessed to handle irregularities, such as missing values and outliers, which are common in real-world datasets. And thereater a extensive EDA is performed to explore the data for underlying patterns and trends using visual and statistical methods. 

The ultimate goal is to deliver a robust predictive model that can be used by stakeholders to anticipate and manage departure delays, thereby minimizing their impact on flight schedules and passenger plans.

In [1]:
#importing all the necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [4]:
# Load dataset
data = pd.read_csv('/kaggle/input/flights-sample-3m/flights_sample_3m.csv')

In [5]:
# Display the first few rows of the dataframe
data.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [6]:
data.columns

Index(['FL_DATE', 'AIRLINE', 'AIRLINE_DOT', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DELAY_DUE_CARRIER',
       'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY',
       'DELAY_DUE_LATE_AIRCRAFT'],
      dtype='object')

In [7]:
# Convert dates to datetime and extract relevant parts
data['FL_DATE'] = pd.to_datetime(data['FL_DATE'])
data['YEAR'] = data['FL_DATE'].dt.year
data['MONTH'] = data['FL_DATE'].dt.month
data['DAY'] = data['FL_DATE'].dt.day
data['WEEKDAY'] = data['FL_DATE'].dt.weekday


In [8]:
data['FL_NUMBER'] = data['FL_NUMBER'].astype(str)

In [9]:
# Remove duplicates
data.drop_duplicates(inplace=True)

In [10]:
# Check missing value of each column
missing_percentage = (data.isnull().sum() / len(data)) * 100
print(missing_percentage)

FL_DATE                     0.000000
AIRLINE                     0.000000
AIRLINE_DOT                 0.000000
AIRLINE_CODE                0.000000
DOT_CODE                    0.000000
FL_NUMBER                   0.000000
ORIGIN                      0.000000
ORIGIN_CITY                 0.000000
DEST                        0.000000
DEST_CITY                   0.000000
CRS_DEP_TIME                0.000000
DEP_TIME                    2.587167
DEP_DELAY                   2.588133
TAXI_OUT                    2.626867
WHEELS_OFF                  2.626867
WHEELS_ON                   2.664800
TAXI_IN                     2.664800
CRS_ARR_TIME                0.000000
ARR_TIME                    2.664733
ARR_DELAY                   2.873267
CANCELLED                   0.000000
CANCELLATION_CODE          97.362000
DIVERTED                    0.000000
CRS_ELAPSED_TIME            0.000467
ELAPSED_TIME                2.873267
AIR_TIME                    2.873267
DISTANCE                    0.000000
D

In [11]:
# Fill missing values for features with low missing percentages with the median
for col in ['DEP_TIME', 'DEP_DELAY', 'ARR_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 
            'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ELAPSED_TIME', 'AIR_TIME']:
    data[col].fillna(data[col].median(), inplace=True)

# Fill missing values for delay reasons with zero
for col in ['DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 
            'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']:
    data[col].fillna(0, inplace=True)

# Fill missing values for CANCELLATION_CODE with 'Not Canceled'
data['CANCELLATION_CODE'].fillna('Not Canceled', inplace=True)

# Check if any more missing values need to be addressed
print(data.isnull().sum())

/tmp/ipykernel_55/1353765903.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipykernel_55/1353765903.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

FL_DATE                     0
AIRLINE                     0
AIRLINE_DOT                 0
AIRLINE_CODE                0
DOT_CODE                    0
FL_NUMBER                   0
ORIGIN                      0
ORIGIN_CITY                 0
DEST                        0
DEST_CITY                   0
CRS_DEP_TIME                0
DEP_TIME                    0
DEP_DELAY                   0
TAXI_OUT                    0
WHEELS_OFF                  0
WHEELS_ON                   0
TAXI_IN                     0
CRS_ARR_TIME                0
ARR_TIME                    0
ARR_DELAY                   0
CANCELLED                   0
CANCELLATION_CODE           0
DIVERTED                    0
CRS_ELAPSED_TIME           14
ELAPSED_TIME                0
AIR_TIME                    0
DISTANCE                    0
DELAY_DUE_CARRIER           0
DELAY_DUE_WEATHER           0
DELAY_DUE_NAS               0
DELAY_DUE_SECURITY          0
DELAY_DUE_LATE_AIRCRAFT     0
YEAR                        0
MONTH     

In [12]:
df_full = df_full = data[['FL_DATE', 'DEP_DELAY', 'ARR_DELAY']].copy()


In [2]:
# Preparing the dataset for EDA
#df_full = pd.read_csv('/kaggle/input/flights-sample-3m/flights_sample_3m.csv', usecols=['FL_DATE', 'DEP_DELAY', 'ARR_DELAY'])


In [13]:
df_full.head()

,FL_DATE,DEP_DELAY,ARR_DELAY
0,2019-01-09,-4.0,-14.0
1,2022-11-19,-6.0,-5.0
2,2022-07-22,6.0,0.0
3,2023-03-06,-1.0,24.0
4,2020-02-23,-2.0,-1.0


In [ ]:
# Converting FL_DATE to datetime and setting it as the index
df_full['FL_DATE'] = pd.to_datetime(df_full['FL_DATE'])
df_full.set_index('FL_DATE', inplace=True)

# Aggregating data to daily average delays
daily_avg_delays = df_full.resample('D').mean()

# Displaying the first few rows to verify the transformation
daily_avg_delays.head()

# EDA - Seaonal Decomposition 

In [ ]:
# Setting the visual style
sns.set(style="whitegrid")

# Plotting the daily average departure and arrival delays
plt.figure(figsize=(15, 7))
plt.plot(daily_avg_delays.index, daily_avg_delays['DEP_DELAY'], label='Average Departure Delay', color='blue')
plt.plot(daily_avg_delays.index, daily_avg_delays['ARR_DELAY'], label='Average Arrival Delay', color='red')
plt.title('Daily Average Flight Delays (2019-2023)')
plt.xlabel('Date')
plt.ylabel('Average Delay (minutes)')
plt.legend()
plt.show()

Resampling the time series data to a daily frequency ('D'), grouping the data by each day and then calculating the mean of each group. 'daily_avg_delays': daily average values of flight delays

In [ ]:
# Decomposing the daily average departure delays
decomposition = seasonal_decompose(daily_avg_delays['DEP_DELAY'], model='additive')

# Plotting the decomposed time series components
fig = decomposition.plot()
fig.set_size_inches(14, 7)
plt.show()

X-axis: data in range 2019 to sometime in 2023. 
Y-axis: the magnitude of the departure delays, trend, seasonal effect, and residuals.
First graph shows the orignial time series data with the daily average departure delays, next one shows the trend of long term progression of the original time series. Third graph shows the seasonality component which in this case is on daily basis and shows a consyant pattern indicadting that as it is plotted against several years of data, daily fluctuations can blend together and look like a solid, unchanging band.The last graph is the spread of the residuals and it seems to be consistent over time, suggesting homoscedasticity, which means the variance of the residuals is fairly uniform. There are a few points that stand out from the zero line, which could be considered as outliers.

In [ ]:

flights_data = pd.read_csv('/kaggle/input/flights-sample-3m/flights_sample_3m.csv', parse_dates=['FL_DATE'])

# Set date as index 
flights_data.set_index('FL_DATE', inplace=True)

# Creating new features
# Time of Day
flights_data['hour_of_day'] = flights_data['CRS_DEP_TIME'].apply(lambda x: int(str(x).zfill(4)[:2]))

# Day of the Week
flights_data['day_of_week'] = flights_data.index.dayofweek

# Month and Season
flights_data['month'] = flights_data.index.month
flights_data['season'] = flights_data['month'] % 12 // 3 + 1

# Weekend Indicator
flights_data['is_weekend'] = flights_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Create categories for distance
flights_data['distance_category'] = pd.cut(flights_data['DISTANCE'], bins=[0, 500, 1000, 1500, 2000, float('inf')], 
                                           labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])

print(flights_data.head())

In [ ]:
# Set the visualisation style
sns.set(style="whitegrid")
 
# EDA: Departure Delays by Hour of Day
plt.figure(figsize=(14, 7))
sns.boxplot(x='hour_of_day', y='DEP_DELAY', data=flights_data)
plt.title('Departure Delays by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Departure Delay (minutes)')
plt.show()



The median delay across most hours seems to be quite low and there is a significant variation in delays, as shown by the range of the boxes and the whiskers for each hour. There are many outliers, especially in the daytime and evening hours, indicating that while most delays are moderate, there are quite a few instances of extreme delays. Certain hours in the evening show a higher median delay, indicating that more than half of the delays recorded during these times are on the higher side compared to other hours.

In [ ]:
# Function to calculate mean and standard error for plotting with error bars
def mean_sem(group):
    return {'mean': group.mean(), 'sem': group.sem()}

# Mean and standard error of departure delays by hour of day
hourly_delay = flights_data.groupby('hour_of_day')['DEP_DELAY'].apply(mean_sem).unstack()

# Plotting mean departure delay by hour of day with error bars
plt.figure(figsize=(14, 7))
plt.errorbar(x=hourly_delay.index, y=hourly_delay['mean'], yerr=hourly_delay['sem'], fmt='o')
plt.title('Mean Departure Delay by Hour of Day with Error Bars')
plt.xlabel('Hour of Day')
plt.ylabel('Mean Departure Delay (minutes)')
plt.grid(True)
plt.show()

The points indicate the average departure delay for flights at different hours of the day. The error bars show the variability around the mean, which can be interpreted as the range within which we expect the true mean delay to fall. They appear to represent one standard error above and below the mean. There's a clear pattern where the mean delay increases during certain hours of the day, peaking in the late afternoon and evening. This suggests that flights are more likely to be delayed during these hours. The variability also seems to increase during the day, with wider error bars observed in the afternoon and evening, indicating less consistency in flight delays during these times.

In [ ]:
# EDA: Departure Delays by Day of Week
plt.figure(figsize=(14, 7))
sns.boxplot(x='day_of_week', y='DEP_DELAY', data=flights_data)
plt.title('Departure Delays by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Departure Delay (minutes)')
# Setting the x-axis labels to be more descriptive
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()


In [ ]:
# Bar plot for mean departure delay by day of the week
plt.figure(figsize=(14, 7))
weekday_mean_delays = flights_data.groupby('day_of_week')['DEP_DELAY'].mean()
sns.barplot(x=weekday_mean_delays.index, y=weekday_mean_delays.values)
plt.title('Average Departure Delay by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Departure Delay (minutes)')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()

There is a consistent pattern of delays throughout the week and no single day stands out as having markedly longer delays.
The mean departure is slightly lower of tuesdays and Wednesdays. 

In [ ]:
# EDA: Departure Delays by Season
plt.figure(figsize=(14, 7))
sns.boxplot(x='season', y='DEP_DELAY', data=flights_data)
plt.title('Departure Delays by Season')
plt.xlabel('Season')
plt.ylabel('Departure Delay (minutes)')
# Setting the x-axis labels to be more descriptive
plt.xticks(ticks=[0, 1, 2, 3], labels=['Winter', 'Spring', 'Summer', 'Autumn'])
plt.show()

In [ ]:
# Bar plot for mean departure delay by season
plt.figure(figsize=(14, 7))
season_mean_delays = flights_data.groupby('season')['DEP_DELAY'].mean()
sns.barplot(x=season_mean_delays.index, y=season_mean_delays.values)
plt.title('Average Departure Delay by Season')
plt.xlabel('Season')
plt.ylabel('Average Departure Delay (minutes)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['Winter', 'Spring', 'Summer', 'Autumn'])
plt.show()


Winter has the highest average delay, which could be attributed to winter weather conditions affecting flight operations. Summer also shows a high average delay, which might be due to increased travel activity. 
Spring and autumn show lower average delays, which might be due to more favorable weather conditions and possibly fewer flights compared to the high travel seasons.

In [ ]:
# EDA: Departure Delays by Distance Category
plt.figure(figsize=(14, 7))
sns.boxplot(x='distance_category', y='DEP_DELAY', data=flights_data)
plt.title('Departure Delays by Distance Category')
plt.xlabel('Distance Category')
plt.ylabel('Departure Delay (minutes)')
plt.show()


The ‘Very Short’ and ‘Very Long’ distance categories have a higher median delay as compared to the other three categories. Additionally, the very long flights also experiencing some of the most extreme delays. 

In [ ]:
plt.figure(figsize=(14, 7))
flights_data['DEP_DELAY'].resample('MS').mean().plot()
plt.title('Monthly Average Departure Delay Over Time')
plt.xlabel('Month')
plt.ylabel('Average Departure Delay (minutes)')
plt.show()


There is some seasonality in the data, with peaks and troughs corresponding to specific times of the year. Starting from around the beginning of 2021, there's a noticeable upward trend in the average departure delay. This trend continues into 2022 and 2023, suggesting that, on average, delays have been getting longer over these years.

In [ ]:
# KDE plot of departure delays
plt.figure(figsize=(14, 7))
sns.histplot(flights_data['DEP_DELAY'].dropna(), bins=50, kde=True)  # Drop NaN values for this plot
plt.title('Distribution of Departure Delays')
plt.xlabel('Departure Delay (minutes)')
plt.ylabel('Frequency')
plt.show()

The distribution is right-skewed, indicating that there are more instances of shorter delays and much fewer instances of longer delays. The long tail extending to the right shows that there are relatively few flights with large delays.

In [ ]:
# Grouping the data by 'AIRLINE' and calculate mean and standard deviation of 'DEP_DELAY'
airline_delays = flights_data.groupby('AIRLINE')['DEP_DELAY'].agg(['mean', 'std']).reset_index()

# Sorting the results for better visualization
airline_delays = airline_delays.sort_values(by='mean')

# Bar plot of average delays by airline
plt.figure(figsize=(14, 7))
sns.barplot(x='mean', y='AIRLINE', data=airline_delays, ci='sd')  # 'ci' for confidence interval, which here means standard deviation
plt.xlabel('Average Delay (minutes)')
plt.ylabel('Airline')
plt.title('Average Departure Delay by Airline')
plt.show()



The specific positioning describes how long is the delay and how predictable are they. Airlines at the top of the chart such as Alaska and Horizon are generally more reliable in terms of having lower and more consistent departure delays, while those at the bottom like Frontier and JetBlue have longer and more unpredictable delay patterns.

In [ ]:
# Grouping the data by 'hour_of_day' and 'AIRLINE', then calculate the mean delay
hourly_airline_delay = flights_data.groupby(['hour_of_day', 'AIRLINE'])['DEP_DELAY'].mean().reset_index()

# Creating a point plot to show the interaction between airline and hour of day for delays
plt.figure(figsize=(18, 10))
sns.pointplot(x='hour_of_day', y='DEP_DELAY', hue='AIRLINE', data=hourly_airline_delay)
plt.title('Average Departure Delay by Airline Across Different Hours of the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Departure Delay (minutes)')
plt.legend(title='Airline', bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend out of the plot
plt.grid(True)
plt.show()


Most airlines seem to have a relatively low and consistent average departure delay across all the hours.

In [ ]:
 # Step 1: Identifying the airlines with the most flights
top_airlines = flights_data['AIRLINE'].value_counts().nlargest(5).index

# Step 2: Filtering the dataset to include only the top airlines
top_airlines_data = flights_data[flights_data['AIRLINE'].isin(top_airlines)]

# Step 3: Calculating the mean delay for these airlines by hour
top_hourly_delay = top_airlines_data.groupby(['hour_of_day', 'AIRLINE'])['DEP_DELAY'].mean().reset_index()

# Step 4: Creating v. vvb                                                      b.    gvv  the visualization
plt.figure(figsize=(18, 10))
sns.pointplot(x='hour_of_day', y='DEP_DELAY', hue='AIRLINE', data=top_hourly_delay, 
              palette='tab10', ci=None)  # ci=None will not show the confidence interval
plt.title('Average Departure Delay by Top Airlines Across Different Hours of the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Departure Delay (minutes)')
plt.legend(title='Airline', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

There is variability in the average delays across different hours for the airlines. Specific hours of concern include early morning 5 am, Midday to early afternoon from 10 am to 2 pm and evening around 8 pm and 9 pm where there is a general increase in delays for most airlines, possibly due to the cumulative effect of delays throughout the day or reduced operational efficiency during late hours.

In [ ]:
flights_data.sort_index(inplace=True)

In [ ]:
flights_data.head()

In [ ]:
print('FL_DATE' in flights_data.columns)

In [ ]:
flights_data = flights_data[~flights_data.index.duplicated(keep='first')]

Creating a subset data to split it into testing and training. The first three years of data will be used for training and the next two for testing the predicted values.

In [ ]:
#subset_data = flights_data['2022-01-01':'2023-12-31'] #test data

In [ ]:
# Defining the split date
split_date = pd.Timestamp('2021-12-31')

# Splitting the data into training and test sets
train_data = flights_data.loc[flights_data.index <= split_date]
test_data = flights_data.loc[flights_data.index > split_date]


# Augmented Dickey-Fuller (ADF) test 

In [ ]:
# Performing the ADF test on the train subset for stationarity
adf_result_subset = adfuller(train_data['DEP_DELAY'].dropna())

print('ADF Statistic: %f' % adf_result_subset[0])
print('p-value: %f' % adf_result_subset[1])
print('Critical Values:')
for key, value in adf_result_subset[4].items():
    print('\t%s: %.3f' % (key, value))

# Interpret the results
if adf_result_subset[1] > 0.05:
    print("The time series subset is likely non-stationary.")
else:
    print("The time series subset is likely stationary.")



Performing ADF on training data to check the stationarity of data. The negative value suggest that the null hypothesis is false, and the data is stationary. Additionally, p-value below the threshold indicates towards rejecting the null hypothesis in favour of the alternative hypothesis that the series is stationary. The ADF statistic here is lower than all the critical value thresholds (1%, 5%, and 10%), further supporting the conclusion that the time series is stationary.

This result confirms that I can apply time series forecasting models that assume stationarity (like ARIMA) on the 'DEP_DELAY' data with some confidence that they are appropriate for this dataset. 


In [ ]:
# ACF and PACF plots
fig, axes = plt.subplots(2, 1, figsize=(12, 12))

# Plot the Autocorrelation Function (ACF)
plot_acf(train_data['DEP_DELAY'].dropna(), lags=50, ax=axes[0])
 
# Plot the Partial Autocorrelation Function (PACF)
plot_pacf(train_data['DEP_DELAY'].dropna(), lags=50, ax=axes[1])

plt.show()


Performing ACF and PACF plots to identify the proper terms and lags to include in time series models for relevant forecasting.

There is a strong autocorrelation at lag 1 for both ACF and PACF which shows that the departure delay ('DEP_DELAY') of a given time is significantly influenced by the departure delay in the immediately preceding time period.


# Modelling

Starting with ARIMA(1, 0, 0) model which corresponds to a simple AR(1) model without the MA component, as there was no significant indication for MA terms from the ACF plot.

# ARIMA (1, 0, 0)

In [ ]:
# Fitting the ARIMA(1, 0, 0) model
arima_model = ARIMA(train_data['DEP_DELAY'], order=(1, 0, 0))
arima_result = arima_model.fit()

# Printing out the summary of the model fit
arima_result_summary = arima_result.summary()
print(arima_result_summary)

# Plotting diagnostics to investigate any unusual behavior
arima_result.plot_diagnostics(figsize=(15, 12))
plt.show()


This model is not a good fit as the coefficient for the AR term is not significant and points towards including additional lags. The non-normality of residuals and substantial deviations in the Q-Q plot suggest that there might be non-linearity or other dynamics in the data. And there is a significant autocorrelation at lag 1 in the correlogram of the residuals which suggests that the model is inefficient in explain the temporal dependencies in the data.  

# ARIMA(1, 0, 1)

In [ ]:
# Fitting the model
arima_model = ARIMA(train_data['DEP_DELAY'], order=(1, 0, 1))
arima_result = arima_model.fit()

# Printing out the summary
arima_result_summary = arima_result.summary()
print(arima_result_summary)


In [ ]:
# Plotting the diagnostics
arima_result.plot_diagnostics(figsize=(15, 12))
plt.show()


In [ ]:
# Getting the number of steps to forecast
steps_to_forecast = len(test_data)

# Generating forecasts
forecast_results = arima_result.get_forecast(steps=steps_to_forecast)


forecast_mean = forecast_results.predicted_mean
forecast_conf_int = forecast_results.conf_int()

In [ ]:
# Plotting the actual values
plt.figure(figsize=(10, 6))
plt.plot(train_data.index, train_data['DEP_DELAY'], label='Train Actuals')
plt.plot(test_data.index, test_data['DEP_DELAY'], label='Test Actuals')

# Plot the predicted values (forecast_mean)
plt.plot(forecast_mean.index, forecast_mean, label='Forecast Mean')

# Plot the confidence intervals (from forecast_conf_int)
plt.fill_between(forecast_conf_int.index,
                 forecast_conf_int['lower DEP_DELAY'],
                 forecast_conf_int['upper DEP_DELAY'],
                 color='pink', alpha=0.3, label='Confidence Interval')

plt.title('Forecast vs Actuals')
plt.xlabel('Date')
plt.ylabel('Departure Delay (minutes)')
plt.legend()
plt.show()


The ARIMA(1, 0, 1) model captures some of the autocorrelation structure of the data which can be concluded by the significant coefficients like 'ar.L1' coefficient is approximately -0.9889 with a p-value of 0.000, which is highly significant and 'ma.L1' coefficient is approximately 0.9765 with a p-value of 0.000, indicating a highly significant moving average component that positively affects the current value. However, the non-normality of the residuals and the spikes in the standardized residuals plot suggest that there may be additional complexity in the data not captured by the model.

# ARIMA (2, 0, 0)

In [ ]:
arima_model_202 = ARIMA(train_data['DEP_DELAY'], order=(2, 0, 2))
arima_result_202 = arima_model_202.fit()

# summary of the model fit
print(arima_result_202.summary())



There are several issues in the model such as non-significant predictors and potential non-stationarity. Thereafter the diagnostic tests indicate good signs in terms of autocorrelation and heteroskedasticity but showcases that there is a problem with the normal distribution of residuals, concluding that the model is not the best fit. 

In [ ]:
# Forecasting using the fitted model
forecast = arima_result_202.get_forecast(steps=len(test_data))

# The forecast object contains the predicted values, the standard error of predictions, and the confidence intervals
forecast_values = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()
forecast_se = forecast.se_mean

In [ ]:
test_data_nonan = test_data.dropna()
forecast_values_nonan = forecast_values.dropna()

In [ ]:
# Ensuring both forecast_values and test_data['DEP_DELAY'] are pandas Series
forecast_values = pd.Series(forecast_values)
test_data['DEP_DELAY'] = pd.Series(test_data['DEP_DELAY'])

# Aligning forecast_values and test_data on their common index
common_index = forecast_values.index.intersection(test_data['DEP_DELAY'].index)
forecast_values_aligned = forecast_values.loc[common_index]
test_data_aligned = test_data['DEP_DELAY'].loc[common_index]

# Plotting the actual vs forecasted values
plt.figure(figsize=(10, 6))
plt.plot(test_data_aligned.index, test_data_aligned, label='Actual', color='blue')
plt.plot(forecast_values_aligned.index, forecast_values_aligned, label='Forecasted', color='red')
plt.title('Actual vs Forecasted DEP_DELAY')
plt.xlabel('Date')
plt.ylabel('DEP_DELAY')
plt.legend()
plt.show()


The forecasted values appear to be relatively flat, which suggests the model might be underfitting the data and not responsive enough to changes over time.


I tried to incorporate the SARIMA model but it is extremely computationally demanding as it tries to include both non-seasonal and seasonal elements, each with its own autoregressive (AR) parameters, differencing (I), and moving average (MA) parameter. Hence choosing a simpler model ETS which is more straightforward and faster to compute than SARIMA models and are particularly useful when you need a model that can be quickly retrained and updated as new data becomes available.

#  Error, Trend, and Seasonality (ETS) model

In [ ]:
flights_data.index = pd.to_datetime(flights_data.index)


In [ ]:
flights_data = flights_data.asfreq('D')  # for daily frequency

In [ ]:
# Making sure that the data does not contain zero or negative values before log transformation
train_data['DEP_DELAY'] = train_data['DEP_DELAY'] + 1  
train_data['Transformed_DEP_DELAY'] = np.log(train_data['DEP_DELAY'])

# Replacing them 
train_data['Transformed_DEP_DELAY'] = train_data['Transformed_DEP_DELAY'].replace([np.inf, -np.inf], np.nan)
train_data.dropna(subset=['Transformed_DEP_DELAY'], inplace=True)

# Fitting the ETS model with log-transformed data
ets_model = ExponentialSmoothing(
    train_data['Transformed_DEP_DELAY'],
    trend='add',
    seasonal='add',
    seasonal_periods=7,  # or another appropriate value based on your domain knowledge
    initialization_method='estimated'
)
ets_result = ets_model.fit()



In [ ]:
# Forecasting using the fitted model
steps_to_forecast = len(test_data)  
predictions = ets_result.forecast(steps=steps_to_forecast)


In [ ]:
# Inversing the transformation for the predictions before comparing with the actual values
predictions = np.exp(predictions)


In [ ]:
# Extracting the residuals
residuals = ets_result.resid

In [ ]:
# Plotting the residuals
plt.figure(figsize=(12, 6))
plt.plot(residuals, label='Residuals', color='blue')
plt.axhline(0, linestyle='--', color='black')  # Add a horizontal line at 0
plt.title('Residuals from ETS Model')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

The graph is shows the error of the model's predictions for flight departure delays over time. Since the residuals are not close to zero and there are spikes corresponding to specific days where delays were particularly unpredictable, we can conclude that the model consistently failed to predict the actual delay and needs to be improved.

In [ ]:
# Additional diagnostic plots
# Histogram of the residuals
plt.figure(figsize=(12, 6))
plt.hist(residuals, bins=25, color='blue', edgecolor='black')
plt.title('Histogram of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

# ACF plot of the residuals
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals, lags=40)
plt.show()

In [ ]:
#MAE and MSE for the training set
mae = np.mean(np.abs(residuals))
mse = np.mean(residuals**2)
print(f"MAE: {mae}, MSE: {mse}")

In [ ]:
# Plotting the actual vs forecasted values
plt.figure(figsize=(10, 4))
plt.plot(test_data.index, test_data['DEP_DELAY'], label='Actual')
plt.plot(test_data.index[-steps_to_forecast:], predictions, label='Forecasted', linestyle='--')
plt.legend()
plt.show()


The model is underperforming and needs some parameters tunning. I was not sure about the spikes on the graph and therefore decided to proceed with anamoly detection and handle those and try the ETS model again. 

# Anomaly detection on the DEP_DELAY column

In [ ]:
# Ensuring that there is no missing values in the 'DEP_DELAY' column
flights_data = flights_data.dropna(subset=['DEP_DELAY'])

# Initializing the IsolationForest model
iforest = IsolationForest(n_estimators=100, contamination=0.01)  # Adjust contamination if you have an expected outlier ratio

# Reshaping the data using .values.reshape(-1, 1) since there is a single feature
iforest.fit(flights_data[['DEP_DELAY']].values.reshape(-1, 1))

# Predicting the anomalies (-1 for outliers, 1 for inliers)
flights_data['anomaly'] = iforest.predict(flights_data[['DEP_DELAY']].values.reshape(-1, 1))

# Filtering out the anomalies to get a DataFrame without outliers
flights_data_no_anomaly = flights_data[flights_data['anomaly'] == 1]



In [ ]:
split_date = pd.Timestamp('2021-12-31')

# Splitting the data into training and test sets
train_data_new = flights_data_no_anomaly.loc[flights_data_no_anomaly.index <= split_date]
test_data_new = flights_data_no_anomaly.loc[flights_data_no_anomaly.index > split_date]

In [ ]:
train_series = train_data_new['DEP_DELAY']

# Fitting the ETS model on the training series
ets_model = ExponentialSmoothing(train_series, trend='add', seasonal='add', seasonal_periods=365)
ets_fit = ets_model.fit()

In [ ]:
# Forecastting the future values using the fitted model
forecast_length = len(test_data_new)
forecast_values = ets_fit.forecast(steps=forecast_length)

# Adding the forecasted values to the test_data DataFrame for comparison
test_data_new['Forecast'] = forecast_values.values

In [ ]:
print(test_data_new['Forecast'])

In [ ]:
forecasted_values_list = test_data_new['Forecast'].tolist()

In [ ]:
count_of_items = len(forecasted_values_list)
print(count_of_items) # as the test_data is for about 2 years or 20 months. 

In [ ]:
test_data_new.head()

In [ ]:
test_data_new.to_csv('test_data_new.csv', index=False) # created this for the group project

In [ ]:
test_data_new['FL_DATE'] = test_data_new.index

plt.figure(figsize=(15, 7))
plt.plot(test_data_new['FL_DATE'], test_data_new['DEP_DELAY'], label='Actual', color='blue')
plt.plot(test_data_new['FL_DATE'], test_data_new['Forecast'], label='Forecasted', color='orange', linestyle='--')

plt.title('Actual vs Forecasted Departure Delays')
plt.xlabel('Date')
plt.ylabel('Departure Delay (minutes)')
plt.legend()
plt.show()


The forecasted values after the anamoly detection and handling is very close to the actual values of the test_data and the model is able to predict very closely to the actual values.

In [ ]:
# Ensuring both are numpy arrays to facilitate calculations
actuals = np.array(test_data_new['DEP_DELAY'])
forecasts = np.array(ets_result.forecast(steps=len(test_data_new)))

# Mean Absolute Error
mae = np.mean(np.abs(forecasts - actuals))

# Mean Squared Error
mse = np.mean((forecasts - actuals) ** 2)

# Root Mean Squared Error
rmse = np.sqrt(mse)

# Symmetric Mean Absolute Percentage Error 
smape = np.mean(2.0 * np.abs(forecasts - actuals) / (np.abs(actuals) + np.abs(forecasts))) * 100

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"SMAPE: {smape}%")


The above error metrics gives more room for imorovement as on average the model’s forecasts are about 15 minutes away from the actual departure delay times. The provided values indicate the model has some predictive capability but is certainly not perfect. The errors suggest that while the forecast can capture the general pattern of departure delays, there are specific instances where it fails to predict accurately and some robust models such as SARIMA can be used for better prediction. 

In [ ]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt


# Plot Autocorrelation
plot_acf(df_full['DEP_DELAY'], lags=30)  # You can adjust the number of lags as needed.
plt.show()

# Plot Partial Autocorrelation
plot_pacf(df_full['DEP_DELAY'], lags=30)  # Adjust the number of lags as needed.
plt.show()


# Limitations

I tried to incorporate a couple of other models as well but they were computationally demanding and there was no way to incorporate those. Some of these models were SARIMA and LSTM. MY forst choice was LSTM but I did not get anywhere using it and hence decided on ARIMA. Whereas due to the computational challenges of SARIMA , I further dived into ETS model. 